## Homework #6

* Due on **Tuesday, March-19-2024**, before class (~1pm)
* Please comment your code and pay attention to plotting, labeling styles
* Upload your code to the following dropbox link given in Moodle

### 1. Write a Python code to perform Monte Carlo simulations for the two-person coin flipping game discussed in class (30%)
- your choice for "head" is 50%，while the other player's choice for "head" is 11/30 (between 1/3 and 2/5), which is within the range we analyzed in class. Show that when you play more and more times, your expectation is negative. 
- now experiment with your code to show that your expectation is largely controlled by the choice of the other player.

### 2. Distributions of GDP and life expectancy (30%)

- Download the data file "gdp_data.txt" from Moodle and load it into Python using Pandas as a data frame
- Print out the head() of your data frame to screen, so you know clearly what column data sets you are working with
- Find out the number of continents in your data set, print out the results (recall in the in-class practice, we used the **.values** property of a column data, or, Series together with the **set()** function, what's a main property of a Python set?)
- Loop through the continents (ignore Oceanic), print out the mean, mode, median and standard deviation of life expectancy for each continent at year 2007
- Which continent(s) has (have) distribution(s) most likely Normal? Why? 
- Make a histogram of the life expectancy in year 2007, divide the data to different subgroups by continents, and compute the skewness of the distributions for each continent
- [**extra credit 10%**] Use the data for all the years (1952-2007) to make a scatter plot showing the relationship between (mean - median), and the skewness of the distributions. 

### 3. Solar wind versus geomagnetic activity  (40%)

Download the data file "OMNI_2003.xls" from Moodle, which is a one-hour solar wind and geomagnetic activity data for the year of 2003. The data file is generated in NASA's cdaweb OMNI data base (https://omniweb.gsfc.nasa.gov/html/ow_data.html), combined from multiple space-based and ground-based measurments. For the interplanetary magnetic field (IMF) components Bx By and Bz:

- X-component pointing from the Earth to the Sun
- Y-component pointing from Dawn to Dusk
- Z-component pointing from the Center of the Earth towards the North Pole

These components basically control the dynamic variation of the space environment. The "KP" index in the data is called the "Planetary-K" index which describe Now let's try some data analysis to learn some space sciences.

- Load the excel file into Python as a Pandas data frame, pay attention to the number of rows you need to skip (use the skiprows = # option)
- Print out all the names of the columns so you know exactly what column data you are working with 
- Create a new column named "AE" (auroral index) by computing (AU - AL), generate a scatter plot to show the relationship between KP and AE. By looking at the plot, what do you think is the relationship between KP and AE?
- What's the maximum value of KP in 2003? What time did the max KP value occur?
- What are the mean, median, max, min values of IMF Bx? 
- Generate a histogram of the interplanetary magnetic field (IMF) Bx, By and Bz, use at least 80 bins
- Test whether the distributions of Bx, By and Bz are Normal or not. Print out the results
- How many modes are there in the Bx data? How many modes are there in the By data? (hint: use the .mode() function)
- Now divide the Bx data into two groups (By>0 and By<0), and use histograms to show the distribution of Bx for each By-group. 
- [**extra credit 10%**] Do your own research and try to explain either one of the following two phenomena: 
    - what happened during the maximum KP time in 2003; or
    - why there are two modes in Bx and By but not Bz.
    
